In [13]:
import cv2
import dlib
import numpy as np

In [14]:
# 육각형 영역에서 원을 검출하여 반환하는 함수
def detect_circle_in_hexagon(img, hexagon):
    # 육각형 좌표를 이용해 마스크 생성
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, np.int32([hexagon]), (255, 255, 255))

    # 마스크 적용
    masked = cv2.bitwise_and(img, mask)
    gray = cv2.cvtColor(masked, cv2.COLOR_BGR2GRAY)
    equalized_image = cv2.equalizeHist(gray)
    blur = cv2.GaussianBlur(equalized_image, (5, 5), 0)
    edges = cv2.Canny(blur, 30, 100)

    # 원 검출
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=20, 
                               minRadius=5, maxRadius=20)
    
    # 원 검출 결과 반환
    return circles

In [15]:
# circles의 정보 입력받아 이미지에 원 둘레를 그리는 함수
def draw_circles(img, circles):
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(img, (x, y), r, (0, 255, 0), 2)

In [16]:
# 얼굴 인식 model, 랜드마크 인식기 생성
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./content/shape_predictor_68_face_landmarks.dat')

# 웹캠 열기
cap = cv2.VideoCapture(0)

while True:
    # 프레임 읽어오기
    ret, frame = cap.read()

    # 얼굴 검출
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        # 얼굴 영역에서 랜드마크 검출
        landmarks = predictor(gray, face)

        # 양 눈 영역을 나타내는 랜드마크 인덱스
        left_eye = [36, 37, 38, 39, 40, 41]
        right_eye = [42, 43, 44, 45, 46, 47]

        # 양 눈의 x, y 좌표를 저장할 리스트 초기화
        left_eye_pts = []
        right_eye_pts = []

        for n in left_eye:
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            left_eye_pts.append((x, y))

        for n in right_eye:
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            right_eye_pts.append((x, y))

        left_circles = detect_circle_in_hexagon(frame, left_eye_pts)
        right_circles = detect_circle_in_hexagon(frame, right_eye_pts)   

        draw_circles(frame, left_circles)
        draw_circles(frame, right_circles)

    # 화면에 프레임 출력
    cv2.imshow("frame", frame)

    # 'q'를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 웹캠 해제 및 창 닫기
cap.release()
cv2.destroyAllWindows()
